
# Personalized Product Recommender System

Este notebook forma parte del pipeline del proyecto y demuestra el flujo de trabajo para:
- preparar los datos
- entrenar y comparar modelos
- seleccionar y persistir el mejor modelo
- generar recomendaciones personalizadas



## 1) Entorno y dependencias

Usaremos `pandas`, `numpy` y `surprise`. Si no los tienes instalados, ejecuta:
```bash
pip install -r ../requirements.txt
```


In [ ]:

# Imports básicos
import pandas as pd
from src.data_loader import load_ratings_csv, surprise_dataset_from_df
from src.model_selection import benchmark, ALGORITHMS
from src.model_training import train_and_save
from src.recommendation_engine import get_top_n_recommendations
print('Imports OK')



## 2) Carga y preprocesado

Coloca tu fichero de ratings en `data/raw/ratings.csv` con columnas `userId,itemId,rating`.
Aquí mostramos cómo cargar una muestra de 1000 filas para experimentación rápida.


In [ ]:

# Ejemplo de carga (si tienes el fichero ratings.csv en data/raw/)
import os
path = os.path.join('..','data','raw','ratings.csv')
if os.path.exists(path):
    df = load_ratings_csv(path, sample=1000)
    print('Muestra cargada:', df.shape)
else:
    print('No se ha encontrado data/raw/ratings.csv — coloca tu fichero allí para ejecutar este notebook.')



## 3) Benchmark de algoritmos

Comparar SVD, KNN y Baseline para seleccionar el mejor algoritmo inicial.


In [ ]:

# Si dispones del dataset cargado anteriormente, ejecuta benchmark
if 'df' in globals():
    data = surprise_dataset_from_df(df)
    results = benchmark(ALGORITHMS, data, cv=3, verbose=False)
    for name, res in results.items():
        print(name, res['test_rmse'].mean(), res['test_mae'].mean())
else:
    print('Dataset no cargado — salta esta celda si solo quieres ver la estructura del notebook.')



## 4) Entrenamiento final y guardado del modelo

Entrenar el modelo seleccionado sobre todo el dataset y guardarlo para producción.


In [ ]:

# Ejemplo de entrenamiento final
from surprise import SVD
import joblib, os
if 'data' in globals():
    full = data.build_full_trainset()
    algo = SVD()
    algo.fit(full)
    os.makedirs('../models/trained', exist_ok=True)
    joblib.dump(algo, '../models/trained/svd_model.pkl')
    print('Modelo entrenado y guardado.')
else:
    print('No hay datos para entrenar en esta sesión.')



## 5) Generar recomendaciones

Uso del motor para obtener top-N para un usuario.


In [ ]:

# Ejemplo de generación de recomendaciones (requiere modelo entrenado y dataset)
if os.path.exists('../models/trained/svd_model.pkl') and 'df' in globals():
    algo = joblib.load('../models/trained/svd_model.pkl')
    # lista de items
    all_items = df['itemId'].unique().tolist()
    trainset = data.build_full_trainset()
    user_id = df['userId'].iloc[0]
    topn = get_top_n_recommendations(algo, trainset, all_items, user_id, n=10)
    print('Top recomendaciones para user', user_id, topn)
else:
    print('Modelo o datos no disponibles para recomendaciones.')
